In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import altair as alt
from altair_saver import save
import os
import re

import numpy as np
import warnings
warnings.simplefilter("ignore")

C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('Data_mesh_publications_cleaned.csv')
df

,Filename,Content,Publisher,Publication_type
0,Breaking Down Data Silos Data Mesh to Achieve ...,abstract data localization law becoming make h...,IEEE,conference
1,Decentralized Data Governance as Part of a Dat...,abstract data socio technical decentralized an...,IEEE,conference
2,Enterprise Data Strategy A Decentralized Data ...,abstract enterprise experience exponential gro...,IEEE,conference
3,Finding Your Way Through the Jungle of Big Dat...,abstract paper present systematic ofcommon ana...,IEEE,conference
4,01. Evolution of Data Architecture,look back data architecture developed response...,Springer,book chapter
5,02. Terminology Data Fabric and Data Mesh,term data fabric data often viewed different c...,Springer,book chapter
6,03. Data Fabric and Data Mesh Use Case Scenarios,organization realized importance implementing ...,Springer,book chapter
7,04. Data Fabric and Data Mesh Business Benefits,defining data fabric underlying data architect...,Springer,book chapter
8,05. Key Data Fabric and Data Mesh Capabilities,introduces key capability concept knowledge ca...,Springer,book chapter
9,06. Relevant ML and DL Concepts,ai ml dlbroadly speaking ai simulate human per...,Springer,book chapter


In [ ]:
# def generate_custom_token_pattern(ngram_min, ngram_max):
#     # Ensure no word repetition in n-grams
#     word_pattern = r'\b(\w+)\b(?=.*\b\1\b)'
    
#     # Combine word patterns for specified n-gram range
#     ngram_pattern = r'(?:' + word_pattern + r'\s){' + str(ngram_min - 1) + r',' + str(ngram_max - 1) + r'}' + word_pattern
    
#     return ngram_pattern

# token_pattern = generate_custom_token_pattern(1, 4)

In [4]:
def extract_topics(text, tfidf_vectorizer, lda_model):
    
    # TF-IDF Vectorization for the specific document
    tfidf_data = tfidf_vectorizer.transform([text])
    tfidf_values = tfidf_data.toarray()
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    df_tfidf = pd.DataFrame(data=tfidf_values, columns=feature_names)

    # Apply LDA
    doc_topic_prob = lda_model.transform(df_tfidf)

    # Get the most probable topic for the document
    most_probable_topic = np.argmax(doc_topic_prob)

    # Get the top words for the most probable topic with TF-IDF values
    topic = lda_model.components_[most_probable_topic]
    top_keywords_idx = topic.argsort()[:-10 - 1:-1]
    top_keywords = [feature_names[i] for i in top_keywords_idx]
    top_tfidf_values = [tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]
#     top_tfidf_values = [tfidf_data[0, tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]


    topic_words = list(zip(top_keywords, top_tfidf_values))

    return most_probable_topic + 1, topic_words


In [5]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4), token_pattern=r'\b\w+\b')
tfidf_data = tfidf_vectorizer.fit_transform(df['Content'])

# Apply LDA
lda = LatentDirichletAllocation(n_components=38, random_state=42)
lda.fit(tfidf_data)


LatentDirichletAllocation(n_components=38, random_state=42)

In [6]:

# Assuming 'df' is your DataFrame with 'pdf_content' column
df['topic'], df['topic_words'] = zip(*df.apply(lambda row: extract_topics(row['Content'], tfidf_vectorizer, lda), axis=1))

In [7]:
df

,Filename,Content,Publisher,Publication_type,topic,topic_words
0,Breaking Down Data Silos Data Mesh to Achieve ...,abstract data localization law becoming make h...,IEEE,conference,31,"[(localization, 3.970414465569701), (data loca..."
1,Decentralized Data Governance as Part of a Dat...,abstract data socio technical decentralized an...,IEEE,conference,26,"[(data, 1.0), (product, 1.137201121513485), (a..."
2,Enterprise Data Strategy A Decentralized Data ...,abstract enterprise experience exponential gro...,IEEE,conference,28,"[(sample vendor, 3.970414465569701), (pak, 2.8..."
3,Finding Your Way Through the Jungle of Big Dat...,abstract paper present systematic ofcommon ana...,IEEE,conference,26,"[(data, 1.0), (product, 1.137201121513485), (a..."
4,01. Evolution of Data Architecture,look back data architecture developed response...,Springer,book chapter,25,"[(edw, 3.0541237336955462), (evolution data, 2..."
5,02. Terminology Data Fabric and Data Mesh,term data fabric data often viewed different c...,Springer,book chapter,23,"[(ai, 1.262364264467491), (metadata, 1.2623642..."
6,03. Data Fabric and Data Mesh Use Case Scenarios,organization realized importance implementing ...,Springer,book chapter,26,"[(data, 1.0), (product, 1.137201121513485), (a..."
7,04. Data Fabric and Data Mesh Business Benefits,defining data fabric underlying data architect...,Springer,book chapter,8,"[(driver, 2.5841201044498106), (key driver, 3...."
8,05. Key Data Fabric and Data Mesh Capabilities,introduces key capability concept knowledge ca...,Springer,book chapter,23,"[(ai, 1.262364264467491), (metadata, 1.2623642..."
9,06. Relevant ML and DL Concepts,ai ml dlbroadly speaking ai simulate human per...,Springer,book chapter,26,"[(data, 1.0), (product, 1.137201121513485), (a..."


# Results with filename

In [ ]:
result_data = []

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    Filename = row['Filename']
    topic_words = row['topic_words']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data.append({'Filename': Filename, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df = pd.DataFrame(result_data, columns=['Filename', 'topic_word', 'idf_value'])

# Print the result DataFrame
result_df

In [ ]:
# Select the top terms for each PDF
top_terms_df = result_df.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df['idf_value'] = top_terms_df['idf_value'] + np.random.rand(top_terms_df.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 2.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda = (heatmap + text).properties(width=2000)


In [ ]:
# save(chart_lda, "lda_idf_heatmap.html")
chart_lda

# Result with publication type

In [ ]:
result_data_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['topic_words']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publication_type.append({'Publication_type': Publication_type, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df_publication_type = pd.DataFrame(result_data_publication_type, columns=['Publication_type', 'topic_word', 'idf_value'])

# Print the result DataFrame
result_df_publication_type

In [ ]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_publication_type.groupby('Publication_type').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['idf_value'] = top_terms_df_publication_type['idf_value'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 2.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publication_type = (heatmap + text).properties(width=2000)
chart_lda_publication_type

# Result with publisher

In [ ]:
result_data_publisher = []

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    Publisher = row['Publisher']
    topic_words = row['topic_words']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publisher.append({'Publisher': Publisher, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df_publisher = pd.DataFrame(result_data_publisher)


result_df_publisher

In [ ]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['idf_value'] = top_terms_df_publisher['idf_value'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 3.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publisher = (heatmap + text).properties(width=1500)
chart_lda_publisher

# USING ONLY TFIDF

In [ ]:
# save(chart_lda_publisher, 'lda_idf_heatmap_pub.html')

In [ ]:
def find_top_words_for_each_document(df, num_top_words=10):

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4), token_pattern=r'\b\w+\b')

    # Fit and transform the PDF content
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

    # Get feature names (words) from the TF-IDF vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame with TF-IDF values
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Now, for each document, find the top words based on TF-IDF scores
    top_words_for_each_document = []
    for i, row in enumerate(tfidf_df.iterrows()):
        _, document_tfidf_scores = row
        top_words_index = document_tfidf_scores.argsort()[-num_top_words:][::-1]
        top_words = [(feature_names[index], document_tfidf_scores[index]) for index in top_words_index]
        top_words_for_each_document.append(top_words)

    # Create a new DataFrame with 'pdf_content' and 'top_words'
    result_df = pd.DataFrame({'Filename': df['Filename'],'Publisher': df['Publisher'],'Publication_type': df['Publication_type'], 'Content': df['Content'], 'top_words': top_words_for_each_document})

    return result_df

In [ ]:
# Call the function
result_df_tfidf = find_top_words_for_each_document(df)

# Display the result DataFrame
# result_df_tfidf['Publisher'] = df['Publisher']
# result_df_tfidf['Publication_type'] = df['Publication_type']
result_df_tfidf

# Results with filename

In [ ]:
result_data_tfidf_filename = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Filename = row['Filename']
    top_words = row['top_words']

    # Iterate through each tuple in the list of top_words
    for word, tfidf in top_words:
        result_data_tfidf_filename.append({'Filename': Filename, 'topic_word': word, 'tfidf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_filename = pd.DataFrame(result_data_tfidf_filename)

result_df_tfidf_filename

In [ ]:
import pandas as pd

# Assuming result_df_3 is your DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)# Show all rows

# Display the entire DataFrame
print(result_df_tfidf_filename)


In [ ]:
# Select the top terms for each PDF
top_terms_df_tfidf = result_df_tfidf_filename.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf['tfidf_value'] = top_terms_df_tfidf['tfidf_value'] + np.random.rand(top_terms_df_tfidf.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_tfidf).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('tfidf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tfidf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.tfidf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf = (heatmap + text).properties(width=2000)


In [ ]:
# save(chart_tfidf, 'tfidf_heatmap.html')
chart_tfidf

# Result with publisher

In [ ]:
result_data_tfidf_publisher = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publisher = row['Publisher']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publisher.append({'Publisher': Publisher, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publisher = pd.DataFrame(result_data_tfidf_publisher)


result_df_tfidf_publisher

In [ ]:
result_df_tfidf_publisher.describe()

In [ ]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_tfidf_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['tf_idf_value'] = top_terms_df_publisher['tf_idf_value'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publisher = (heatmap + text).properties(width=1500)
chart_publisher

In [ ]:
# save(chart_tfidf_publisher_2, 'tfidf_heatmap_publisher.html')

# Results with Publisher

In [ ]:
result_data_tfidf_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publication_type.append({'Publication_type': Publication_type, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publication_type = pd.DataFrame(result_data_tfidf_publication_type)


result_df_tfidf_publication_type

In [ ]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_tfidf_publication_type.groupby('Publication_type').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['tf_idf_value'] = top_terms_df_publication_type['tf_idf_value'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publication_type = (heatmap + text).properties(width=1500)
chart_publication_type

In [9]:
from gensim import corpora, models
from gensim.utils import simple_preprocess


df['Content'] = df['Content'].astype(str)


document_topics = []


for index, row in df.iterrows():

    text = simple_preprocess(row['Content'], deacc=True)
    
    # Create a dictionary and corpus for the document
    dictionary = corpora.Dictionary([text])
    corpus = [dictionary.doc2bow(text)]
    
 
    lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, passes=15, random_state=100)
    

    topics = lda_model.print_topics(num_words=3)
    topics_str = ''.join([f"[Topic {topic[0]}: {', '.join(topic[1].split('+')).replace('*','').strip()}]" for topic in topics])
    

    document_topics.append((row['Filename'], topics_str))


In [8]:
topics_df = pd.DataFrame(document_topics, columns=['Filename', 'Topics'])
topics_df

,Filename,Topics
0,Breaking Down Data Silos Data Mesh to Achieve ...,"[Topic 0: 0.003""data"" , 0.002""localization"" ,..."
1,Decentralized Data Governance as Part of a Dat...,"[Topic 0: 0.063""data"" , 0.030""product"" , 0.0..."
2,Enterprise Data Strategy A Decentralized Data ...,"[Topic 0: 0.039""data"" , 0.007""product"" , 0.0..."
3,Finding Your Way Through the Jungle of Big Dat...,"[Topic 0: 0.007""data"" , 0.004""architecture"" ,..."
4,01. Evolution of Data Architecture,"[Topic 0: 0.143""data"" , 0.035""architecture"" ,..."
5,02. Terminology Data Fabric and Data Mesh,"[Topic 0: 0.004""data"" , 0.002""fabric"" , 0.00..."
6,03. Data Fabric and Data Mesh Use Case Scenarios,"[Topic 0: 0.006""data"" , 0.002""fabric"" , 0.00..."
7,04. Data Fabric and Data Mesh Business Benefits,"[Topic 0: 0.009""data"" , 0.009""architecture"" ,..."
8,05. Key Data Fabric and Data Mesh Capabilities,"[Topic 0: 0.006""data"" , 0.004""metadata"" , 0...."
9,06. Relevant ML and DL Concepts,"[Topic 0: 0.001""data"" , 0.001""ml"" , 0.001""ai..."
